<a href="https://colab.research.google.com/github/tsachiblau/sparseGANProject/blob/master/GAN_FULL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

parameters setup

In [1]:
from keras.optimizers import Adam
import numpy as np

np.random.seed(10)
# params
noise_dim = 100
batch_size = 16
half_batch_size = int(batch_size/2)
steps_per_epoch = 3750  # batch_size * steps_per_epoch = epoch size (train size)
epochs = 10
img_rows, img_cols, channels = 28, 28, 1
sparse_dim = img_rows * img_cols * channels
L1_weight = 10   # Loss = data_fidelity + L1_weight * L1_norm
thr_value = 0.1 # x = |x| >= thr_value ? x : 0

# optimizer - LR and Beta1 are hyper parameters
optimizer = Adam(0.0002,0.5)
optimizer2 = Adam(1e-6)
train_only_fours = False  # overfit to 4 creation - sanity check
warm_start_discriminator = False # init discriminator by training it *a little bit* as classifier
load_dict = True  #load pretrained dictionary
use_noisy_labels = False # use soft labels and not 0,1 to train
use_reconstructor = False
#
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Using TensorFlow backend.


Thu Mar 19 11:37:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

import data set + preprocessing

In [2]:
from keras.datasets import mnist
from keras.utils import np_utils
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train.astype(np.float32) - 127.5) / 127.5  # normalize between +1 -1

# take only digit 4
x_four_digit = x_train[y_train==4]
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

x_train = x_train.reshape(-1, img_rows*img_cols*channels) # each image as vector
x_four_digit = x_four_digit.reshape(-1, img_rows*img_cols*channels) # each image as vector
x_test = x_test.reshape(-1, img_rows*img_cols*channels) # each image as vector
np.random.shuffle(x_train)
np.random.shuffle(x_four_digit)
print(x_train.shape)
print(x_four_digit.shape)

11493376/11490434 [==============================] - 1s 0us/step
(60000, 784)
(5842, 784)


Load dictionary from pickle file

In [3]:
import matplotlib.pyplot as plt
from sklearn.decomposition import DictionaryLearning
import pickle
import os.path
from os import path

#dictionary file name
file_name = 'dictionary20.pkl'

#check if dictionary exists
if not path.exists(file_name):
  
  d=DictionaryLearning(n_components=sparse_dim, max_iter=10)
  # train dictionary
  d.fit(x_train[1:10000, :])
  dictionary_loaded2 = np.transpose(d.components_)
  print(dictionary.shape)

  with open(file_name, 'wb') as output:
    pickle.dump(d, output, pickle.HIGHEST_PROTOCOL)
  print("created new dictionary")

else:
  with open(file_name, 'rb') as input:
    d = pickle.load(input)
    dictionary_loaded2 = np.transpose(d.components_)
  # print(d)
  print("loaded dictionary")

loaded dictionary


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictionaryLearning from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


get transformed data set

In [0]:
from numpy import savetxt
transformed_x = d.transform(x_train)
transformed_x.shape


In [4]:
savetxt('transformed_x.csv', transformed_x, delimiter=',')

NameError: ignored

Create NN modules

In [0]:
# with pursuit and thresholding approach
from keras.utils import plot_model
from keras.models import Sequential
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Input
from keras.models import Model
from keras.initializers import RandomNormal, glorot_uniform
from keras.layers import Dense, Conv2D, Flatten, Reshape, Dropout,AveragePooling2D, MaxPooling2D, BatchNormalization, Lambda, ReLU, ThresholdedReLU, Activation, GaussianNoise
from tensorflow.linalg import matmul, matrix_transpose
from tensorflow.math import abs
from keras import regularizers
from keras.losses import kullback_leibler_divergence, binary_crossentropy
from keras.utils.generic_utils import get_custom_objects
import tensorflow as tf
from tensorflow import cast
from keras.activations import sigmoid
from keras.utils import np_utils
from keras.backend import random_normal,shape,exp


# costum loss - TODO:: parameter to TUNE
def gaussian_prob_loss(y_actual,y_pred):
  probs = 1 - K.square(K.exp(-0.5*K.square(y_pred)))    #1 - prob of y_pred to be drawn from gaussian with mu = 0, std = 1
  return probs



# would cause small entries in absulute values to be set to zero and will create sparse vectors
def soft_threshlding(x):
  above_thr_pos = x * cast(x >= thr_value, tf.float32)
  below_thr_neg = x * cast(x <= -thr_value, tf.float32)
  return (above_thr_pos + below_thr_neg)



get_custom_objects().update({'soft_threshlding': Activation(soft_threshlding)})



# classifier
def create_trained_classifier():
    classifier = Sequential()
    #16 - better
    classifier.add(Conv2D(16, (3, 3), strides=(2,2),padding='same', kernel_regularizer=regularizers.l2(0.01),kernel_initializer=RandomNormal(0, 0.02), input_shape=(img_cols, img_rows, channels),name="input"))
    #classifier.add(BatchNormalization(name="BN0"))
    classifier.add(LeakyReLU(0.2,name="Activation0"))
    #32 - better
    classifier.add(Conv2D(32, (3, 3), strides=(2,2), padding='same',  kernel_regularizer=regularizers.l2(0.01),kernel_initializer=RandomNormal(0, 0.02),name="conv1"))
    #classifier.add(BatchNormalization(name="BN1"))
    classifier.add(LeakyReLU(0.2,name="Activation1"))
    # 64 - better
    classifier.add(Conv2D(64, (3, 3), strides=(2,2), padding='same',  kernel_regularizer=regularizers.l2(0.01),kernel_initializer=RandomNormal(0, 0.02),name="conv2"))
    #classifier.add(BatchNormalization(name="BN2"))
    classifier.add(LeakyReLU(0.2,name="Activation2"))
    
    classifier.add(Flatten(name="Flatten"))
    classifier.add(Dropout(0.4,name="dropout"))
    classifier.add(Dense(10, activation='softmax',kernel_regularizer=regularizers.l2(0.01),kernel_initializer=RandomNormal(0, 0.02),name="Dense"))   # 1 number which indicates real / fake
    classifier.trainable = True
    classifier.compile(loss='categorical_crossentropy', optimizer=optimizer)

    model=classifier.fit(x_train.reshape(-1,img_rows,img_cols,channels), y_train, 
                              validation_data=(x_test.reshape(-1,img_rows,img_cols,channels), y_test), epochs=5, batch_size=64)
    return classifier
# input: image size
# output: 1 number - 1 real 0 fake
# descriminator:
def create_discriminator():
  if(warm_start_discriminator):
    classifier = create_trained_classifier()
    classifier.pop()
    classifier.add(Dense(1, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01),kernel_initializer=RandomNormal(0, 0.02),name="Dense"))   # 1 number which indicates real / fake
    classifier.compile(loss='binary_crossentropy', optimizer=optimizer)
    return classifier
  else:
    discriminator = Sequential()
    #16 - better
    discriminator.add(Conv2D(16, (3, 3), strides=(2,2),padding='same', kernel_regularizer=regularizers.l2(0.01),kernel_initializer=RandomNormal(0, 0.02), input_shape=(img_cols, img_rows, channels),name="input"))
    #discriminator.add(BatchNormalization(name="BN0"))
    discriminator.add(LeakyReLU(0.2,name="Activation0"))
    #32 - better
    discriminator.add(Conv2D(32, (3, 3), strides=(2,2), padding='same',  kernel_regularizer=regularizers.l2(0.01),kernel_initializer=RandomNormal(0, 0.02),name="conv1"))
    #discriminator.add(BatchNormalization(name="BN1"))
    discriminator.add(LeakyReLU(0.2,name="Activation1"))
    # 64 - better
    discriminator.add(Conv2D(64, (3, 3), strides=(2,2), padding='same',  kernel_regularizer=regularizers.l2(0.01),kernel_initializer=RandomNormal(0, 0.02),name="conv2"))
    #discriminator.add(BatchNormalization(name="BN2"))
    discriminator.add(LeakyReLU(0.2,name="Activation2"))
    
    discriminator.add(Flatten(name="Flatten"))
    discriminator.add(Dropout(0.4,name="dropout"))
    discriminator.add(Dense(1, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01),kernel_initializer=RandomNormal(0, 0.02),name="Dense"))   # 1 number which indicates real / fake
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return discriminator


# input: sparse vec size
# output: noise size
def create_encoder():
    reconstructor = Sequential()
    init = RandomNormal(0,stddev=0.02)

    reconstructor.add(Dense(512,  kernel_initializer=init, input_dim=(img_rows*img_cols*channels),name="rec_input"))
    reconstructor.add(BatchNormalization(name="BN0"))
    reconstructor.add(LeakyReLU(0.2,name="activation0"))

    reconstructor.add(Dense(256, kernel_initializer=init,name="hidden1"))
    reconstructor.add(BatchNormalization(name="BN1"))
    reconstructor.add(LeakyReLU(0.2,name="activation1"))

    reconstructor.add(Dense(noise_dim, kernel_initializer=init,name="hidden2"))
    reconstructor.add(BatchNormalization(name="BN2"))
    return reconstructor

# input: noise size
# output: sparse vec size
def create_generator():
    generator = Sequential()
    init = RandomNormal(0,stddev=0.02)
    generator.add(Dense(256, kernel_initializer=init, input_dim=noise_dim,name="init"))
    generator.add(BatchNormalization(name="BN1"))
    generator.add(LeakyReLU(0.2,name="Activation1"))

    generator.add(Dense(512,kernel_initializer=init,name="hidden1"))
    generator.add(BatchNormalization(name="BN2"))
    generator.add(LeakyReLU(0.2,name="Activation2"))

    # generator.add(Dense(1024,kernel_initializer=init,name="hidden2"))
    # generator.add(BatchNormalization(name="BN3"))
    # generator.add(LeakyReLU(0.2,name="Activation3"))

    generator.add(Dense(sparse_dim, kernel_initializer=init,name="hidden3"))
    # soft thresholding activation
    generator.add(BatchNormalization(name="BN4"))
    generator.add(Activation(soft_threshlding, name='soft_threshlding'))
    return generator


generator = create_generator()
encoder = create_encoder()
discriminator = create_discriminator()
# naming:
generator.name = "Gen"
encoder.name = "Rec"
discriminator.name = "Disc"
#
discriminator.summary()


create GAN

In [0]:
def create_gan(discriminator, generator):
    discriminator.trainable=False
    gan_input = Input(shape=(noise_dim,))
    # G(z) is a sparse vector
    sparse_vec = generator(gan_input)   # sparse_dim size - it is thresholded
    # a layer that creates the multiplication between the sparse tensor with dictionary to create image
    fake_image_vec = Lambda(lambda x: matrix_transpose(matmul(dictionary_loaded2,matrix_transpose(x))),name='mul_in_D')(sparse_vec)
    # reshape to image size - will be inserted to discriminator
    fake_image = Reshape((img_rows, img_cols, channels),name="Reshape_as_im")(fake_image_vec)
    gan_output = discriminator(fake_image)  # the descriminator output
    # todo - double output double loss
    gan = Model(gan_input, [gan_output,sparse_vec])
    gan.compile(loss=['binary_crossentropy','mean_absolute_error'], optimizer=optimizer,loss_weights=[1,L1_weight])
    plot_model(gan, to_file='model_plot.png', show_shapes=True, show_layer_names=True,expand_nested=True)
    return gan

gan = create_gan(discriminator,generator)
print(gan.summary())

print(discriminator.summary())
print(discriminator.layers[0].trainable)
print(gan.get_layer(name="Disc").trainable)
print(gan.get_layer(name="Gen").trainable)


create AE module

In [0]:
# define ae
vae_input = Input(shape=(sparse_dim,))
encoder_output = encoder(vae_input)
# G(z) is a sparse vector
sparse_vec2 = generator(encoder_output)   # sparse_dim size - it is thresholded
# a layer that creates the multiplication between the sparse tensor with dictionary to create image
fake_image_vec2 = Lambda(lambda x: matrix_transpose(matmul(dictionary_loaded2,matrix_transpose(x))),name='mul_in_D')(sparse_vec2)

vae = Model(vae_input, [encoder_output, sparse_vec2,fake_image_vec2])  #todo add L2 on image
vae.compile(loss=[gaussian_prob_loss,'mean_absolute_error','mean_squared_error'],loss_weights=[0.1,1,1],optimizer=optimizer)
plot_model(vae, to_file='model_plot2.png', show_shapes=True, show_layer_names=True,expand_nested=True)

Train AE sanity check:

In [0]:
# sanity check
from numpy.random import uniform
import time

t = time.time()
zeros = np.zeros(60000*100)
zeros = zeros.reshape(60000,100)
vae.fit(transformed_x,[zeros,transformed_x,x_train],epochs = 20, shuffle=True)
print("elapsed ",time.time()-t)
def show_images2():
    noise = np.random.normal(0, 1, size=(100, 100))
    generated_images = generator.predict(noise)
    generated_images = np.matmul(np.transpose(d.components_),np.transpose(generated_images))
    generated_images = np.transpose(generated_images)
    plt.figure(figsize=(10, 10))
    for i, image in enumerate(generated_images):
        plt.subplot(10, 10, i+1)
        if channels == 1:
            plt.imshow(image.reshape((img_rows, img_cols)), cmap='gray')
        else:
            plt.imshow(image.reshape((img_rows, img_cols, channels)))
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()



idx = int(uniform(0,59999))

im = np.array(x_train[idx,:])
im = im.reshape(28,28)
plt.imshow(im,cmap="gray")
plt.show()

vae_pred = vae.predict(transformed_x[idx:idx+1,:])[2]
im_r = vae_pred[0].reshape(28,28)
plt.imshow(im_r,cmap="gray")
plt.show()


# gen new data

show_images2()





train GAN sanity check:

In [0]:
from numpy.random import uniform

def d_ratio_on_fake(x):
  return np.sum(discriminator.predict(x))/x.shape[0]

def generate_fake_samples(size):
  noise = np.random.normal(0, 1, size=(size, noise_dim))  # will generate batch amount of noise vectors (gaussian distribution)    
  fake_x = generator.predict(noise) # create G(z)
  # fake is (size,sparse_dim)
  fake_x = np.transpose(fake_x)
  # multiply by dict
  fake_x = np.matmul(dictionary_loaded2,fake_x)
  fake_x = np.transpose(fake_x)
  fake_x = fake_x.reshape(size,img_rows,img_cols,channels)
  return fake_x

def generate_real_samples(size):
  real_idx = np.random.randint(0, x_train.shape[0], size=half_batch_size)
  real_x = x_train[real_idx] # sample real image
  real_x = real_x.reshape(fake_x.shape)
  return real_x

def plot_statistics(descriminator_real_probability, descriminator_fake_probability, descriminator_real_loss, descriminator_fake_loss):
  plt.figure()
  plt.subplot(131)
  plt.title('discriminator probability')
  plt.plot(iters, descriminator_real_probability)
  plt.plot(iters, descriminator_fake_probability)
  plt.legend(['real', 'fake'])

  plt.subplot(132)
  plt.title('discriminator loss')
  plt.plot(iters, descriminator_real_loss)
  plt.plot(iters, descriminator_fake_loss)
  plt.legend(['real', 'fake'])
  plt.show()

def show_images(noise):
    generated_images = generator.predict(noise)
    generated_images = np.matmul(dictionary_loaded2,np.transpose(generated_images))
    generated_images = np.transpose(generated_images)
    plt.figure(figsize=(10, 10))
    for i, image in enumerate(generated_images):
        plt.subplot(10, 10, i+1)
        if channels == 1:
            plt.imshow(image.reshape((img_rows, img_cols)), cmap='gray')
        else:
            plt.imshow(image.reshape((img_rows, img_cols, channels)))
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

def label_smoothing(y):  #recieves arrray of ones / zeros and smooth it randomly
  return (y + (random(y.shape) * 0.3))

stop = False
fake_x = 0

descriminator_real_probability = np.array([])
descriminator_fake_probability = np.array([])

descriminator_fake_loss = np.array([])
descriminator_real_loss = np.array([])

reconstructor_loss_list = np.array([])

iters= np.array([])
n_d_real_list = np.array([])
n_d_fake_list = np.array([])

d_loss_real = 0 
d_loss_fake = 0
g_loss = 0

n_d = 10 # 1 10 100
n_g = 10  # 1 10 100
n_r = 1  # 1 10 100



for epoch in range(epochs):  #epochs
  for iter in range(int(steps_per_epoch)):
    i = iter + int(epoch*(steps_per_epoch))

    #discriminator.trainable = True
    # train D_fake
    for batch in range(n_d):  #steps_per_epoch
        fake_x = generate_fake_samples(half_batch_size)
        # fake_x is of size (half_batch_size,rows,cols,channels)
        disc_y_fake = uniform(0,0.2,half_batch_size)
        # train_on_batch recieves data and desired output
        d_loss_fake = discriminator.train_on_batch(fake_x, disc_y_fake)
        real_x =generate_real_samples(half_batch_size)
        disc_y_real = uniform(0.7,1.2,half_batch_size)
        # train_on_batch recieves data and desired output
        d_loss_real = discriminator.train_on_batch(real_x, disc_y_real)

    #discriminator.trainable = False
    # train G
    for batch in range(n_g):  #steps_per_epoch
        noise = np.random.normal(0, 1, size=(batch_size, noise_dim))  # will generate batch amount of noise vectors (gaussian distribution)
        # train GAN (descriminator weights are fixed)
        y_gen_1 = uniform(0.7,1,batch_size)
        y_gen_2 = np.zeros(batch_size * sparse_dim)
        y_gen_2_reshape = y_gen_2.reshape(batch_size,sparse_dim)
        g_loss = gan.train_on_batch(noise, [y_gen_1, y_gen_2_reshape])

    ########################################################## statistics ########################################################## 
    descriminator_fake_loss = np.append(descriminator_fake_loss ,d_loss_fake)        
    descriminator_real_loss = np.append(descriminator_real_loss ,d_loss_real)        

    real_data = generate_real_samples(batch_size)
    fake_data = generate_fake_samples(batch_size)
    descriminator_fake_probability = np.append(descriminator_fake_probability ,d_ratio_on_fake(fake_data))        
    descriminator_real_probability = np.append(descriminator_real_probability ,d_ratio_on_fake(real_data))        
    iters =  np.append(iters,i)
    if(i%10==0):
      plot_statistics(descriminator_real_probability, descriminator_fake_probability, descriminator_real_loss, descriminator_fake_loss)
      noise = np.random.normal(0, 1, size=(10, noise_dim))
      show_images(noise)



  print(f'Epoch: {epoch + 1} \t Discriminator Loss_fake: {d_loss_fake} \t Discriminator Loss_real: {d_loss_real}\t\t Generator Loss: {g_loss}')
  plot_statistics(descriminator_real_probability, descriminator_fake_probability, descriminator_real_loss, descriminator_fake_loss)

  noise = np.random.normal(0, 1, size=(100, noise_dim))
  show_images(noise)
  

Combined training

In [0]:
from numpy.random import uniform

def d_ratio_on_samples(x):
  return np.sum(discriminator.predict(x))/x.shape[0]

def generate_fake_samples(size):
  noise = np.random.normal(0, 1, size=(size, noise_dim))  # will generate batch amount of noise vectors (gaussian distribution)    
  fake_x = generator.predict(noise) # create G(z)
  # fake is (size,sparse_dim)
  fake_x = np.transpose(fake_x)
  # multiply by dict
  fake_x = np.matmul(dictionary_loaded2,fake_x)
  fake_x = np.transpose(fake_x)
  fake_x = fake_x.reshape(size,img_rows,img_cols,channels)
  return fake_x

def generate_real_samples(size):
  real_idx = np.random.randint(0, x_train.shape[0], size=half_batch_size)
  real_x = x_train[real_idx] # sample real image
  real_x = real_x.reshape(fake_x.shape)
  return real_x

def plot_statistics(descriminator_real_probability, descriminator_fake_probability):
  plt.figure()
  plt.title('discriminator probability')
  plt.plot(iters, descriminator_real_probability)
  plt.plot(iters, descriminator_fake_probability)
  plt.legend(['real', 'fake'])
  plt.show()

def show_images(noise):
    generated_images = generator.predict(noise)
    generated_images = np.matmul(dictionary_loaded2,np.transpose(generated_images))
    generated_images = np.transpose(generated_images)
    plt.figure(figsize=(10, 10))
    for i, image in enumerate(generated_images):
        plt.subplot(10, 10, i+1)
        if channels == 1:
            plt.imshow(image.reshape((img_rows, img_cols)), cmap='gray')
        else:
            plt.imshow(image.reshape((img_rows, img_cols, channels)))
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()


descriminator_real_probability = np.array([])
descriminator_fake_probability = np.array([])
iters= np.array([])

# training amount params
n_d = 30 # 1 10 100
n_g = 10  # 1 10 100
n_r = 10  # 1 10 100
i = 0

for epoch in range(epochs):  #epochs
  for iter in range(int(steps_per_epoch)):
    i += 1
    # train D
    for batch in range(n_d):  #steps_per_epoch
        fake_x = generate_fake_samples(half_batch_size)
        # fake_x is of size (half_batch_size,rows,cols,channels)
        disc_y_fake = uniform(0,0.2,half_batch_size)
        # train_on_batch recieves data and desired output
        d_loss_fake = discriminator.train_on_batch(fake_x, disc_y_fake)
        real_x =generate_real_samples(half_batch_size)
        disc_y_real = uniform(0.7,1.2,half_batch_size)
        # train_on_batch recieves data and desired output
        d_loss_real = discriminator.train_on_batch(real_x, disc_y_real)

    # train G
    for batch in range(n_g):  #steps_per_epoch
        noise = np.random.normal(0, 1, size=(batch_size, noise_dim))  # will generate batch amount of noise vectors (gaussian distribution)
        # train GAN (descriminator weights are fixed)
        y_gen_1 = uniform(0.7,1,batch_size)
        y_gen_2 = np.zeros(batch_size * sparse_dim)
        y_gen_2_reshape = y_gen_2.reshape(batch_size,sparse_dim)
        g_loss = gan.train_on_batch(noise, [y_gen_1, y_gen_2_reshape])

    # train R
    for batch in range(n_r):  #steps_per_epoch
      rand_idxs = np.random.randint(0, x_train.shape[0], size=64)
      x = x_train[rand_idxs] # sample real transformed image
      trans_x = transformed_x[rand_idxs]
      zeros = np.zeros(64*noise_dim)
      zeros = zeros.reshape(64,noise_dim)
      labels = [zeros,trans_x,x]
      r_loss = vae.train_on_batch(trans_x,labels)

    ########################################################## statistics ##########################################################       

    real_data = generate_real_samples(batch_size)
    fake_data = generate_fake_samples(batch_size)
    descriminator_fake_probability = np.append(descriminator_fake_probability ,d_ratio_on_samples(fake_data))        
    descriminator_real_probability = np.append(descriminator_real_probability ,d_ratio_on_samples(real_data))        
    iters =  np.append(iters,i)
    if(i%100==0):
      plot_statistics(descriminator_real_probability, descriminator_fake_probability)
      noise = np.random.normal(0, 1, size=(10, noise_dim))
      show_images(noise)



  print(f'Epoch: {epoch + 1} \t Discriminator Loss_fake: {d_loss_fake} \t Discriminator Loss_real: {d_loss_real}\t\t Generator Loss: {g_loss}')
  plot_statistics(descriminator_real_probability, descriminator_fake_probability, descriminator_real_loss, descriminator_fake_loss)

  noise = np.random.normal(0, 1, size=(100, noise_dim))
  show_images(noise)
  